Normalizando os dados

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path_hist = 'C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\data\\hist_cotacoes\\'
lista_historicos = os.listdir(path_hist)
# print(lista_historicos)

In [3]:
# Removendo tickers inexistentes
lista_tickers_removidos = ["APTI4", "BPAR3", "MMAQ4", "MMAQ3"]
lista_hist_removidos = [f"hist_{x}.csv" for x in lista_tickers_removidos]

for historico in lista_historicos:

    # print(path_hist + historico)
    if historico in lista_hist_removidos:
        os.remove(path_hist + historico)

In [4]:
lista_tickers_removidos = ["APTI4", "BPAR3", "MMAQ4", "MMAQ3"]
lista_hist_removidos = [f"hist_{x}.csv" for x in lista_tickers_removidos]
elementos_comuns = [elemento for elemento in lista_hist_removidos if elemento in os.listdir(path_hist)]

print(elementos_comuns)

[]


In [5]:
for historico in lista_historicos:
    df = pd.read_csv(path_hist + historico, index_col="Date")
    df["Adj Close"] = df["Adj Close"]/df.iloc[0]["Adj Close"]
    df.to_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\data\\hist_cot_norm\\{historico}", index=True)

Agrupando os ações por segmento de atuação

In [6]:
# Listando os segmentos de mercado
df_segm_merc_empr = pd.read_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\data\\info_empresas.csv")

for ticker_removido in lista_tickers_removidos:
    df_segm_merc_empr = df_segm_merc_empr[df_segm_merc_empr["tickers"] != ticker_removido]
    
lista_segmentos = df_segm_merc_empr["segmento"].unique().tolist()

In [7]:
df_hist_cot_ticker = pd.read_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\data\\hist_cot_norm\\hist_WEGE3.csv",
                                  index_col="Date")
df_hist_cot_ticker.index = pd.to_datetime(df_hist_cot_ticker.index)

In [8]:
for segmento in lista_segmentos:

    lista_tickers_segm = [x for x in df_segm_merc_empr[df_segm_merc_empr["segmento"] == segmento]["tickers"]]
    # print(lista_tickers_segm)

    data_inicial = '2013-01-01'
    data_final = '2023-10-01'
    datas = pd.date_range(start=data_inicial, end=data_final, freq='D')
    df_segm = pd.DataFrame({'Date': datas}).set_index('Date')

    for ticker in lista_tickers_segm:
        
        df_hist_cot_ticker = pd.read_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\data\\hist_cot_norm\\hist_{ticker}.csv",
                                  index_col="Date")
        # df_hist_cot_ticker = pd.read_csv(
        #    f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\data\\hist_cot_norm\\hist_{ticker}.csv",
        #    parse_dates= "Date",
        #    index_col="Date")
        
        for data in df_hist_cot_ticker.index:
          df_segm.loc[data, ticker] = df_hist_cot_ticker.loc[data, "Adj Close"]

    # df_segm = df_segm.dropna(how="all") # para deletar apenas as linhas vazias
    # df_segm = df_segm.dropna() # para deletar todas as linhas com algum valor vazio

    df_segm.to_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\data\\hist_cot_norm_agrup_segm\\{segmento}.csv")

Gerando um df com o histórico de variação média de cada segmento

In [9]:
data_inicial = '2013-01-01'
data_final = '2023-10-01'
datas = pd.date_range(start=data_inicial, end=data_final, freq='D')
df_media_segm = pd.DataFrame({'Date': datas}).set_index('Date')

for segmento in lista_segmentos:

    df_hist_cot_ticker = pd.read_csv(
        f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\data\\hist_cot_norm_agrup_segm\\{segmento}.csv", 
        index_col="Date")
    # df_hist_cot_ticker = pd.read_csv(
    #     f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\data\\hist_cot_norm_agrup_segm\\{segmento}.csv", 
    #     parse_dates= "Date",
    #     index_col="Date")
    
    for data in df_hist_cot_ticker.index:
        
        soma_linha = 0
        cont_tickers_somados = 0
        
        for ticker in df_hist_cot_ticker.columns:
            
            cond_calc_media = df_hist_cot_ticker.loc[data, ticker] !=0 and not np.isnan(df_hist_cot_ticker.loc[data, ticker])
            if cond_calc_media:
                
                soma_linha += df_hist_cot_ticker.loc[data, ticker]
                cont_tickers_somados += 1
        
        if cont_tickers_somados > 0:
            df_hist_cot_ticker.loc[data, "media"] = soma_linha / cont_tickers_somados 
    
    for data in df_hist_cot_ticker.index:
        df_media_segm.loc[data, segmento] = df_hist_cot_ticker.loc[data, "media"]

In [10]:
df_media_segm.head(10)

,"Exploração, Refino e Distribuição",Equipamentos e Serviços,Minerais Metálicos,Siderurgia,Artefatos de Ferro e Aço,Artefatos de Cobre,Petroquímicos,Fertilizantes e Defensivos,Químicos Diversos,Madeira,...,Bancos,Soc. Crédito e Financiamento,Serviços Financeiros Diversos,Seguradoras,Resseguradoras,Corretoras de Seguros e Resseguros,Exploração de Imóveis,Intermediação Imobiliária,Holdings Diversificadas,Outros
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.0
2013-01-03,1.015322,1.028559,0.994528,1.010880,1.000000,0.936759,1.0,1.006082,1.000000,0.995308,...,1.014291,1.000000,1.010567,1.004237,NaN,1.012595,1.001772,1.001923,0.993357,1.0
2013-01-04,1.022497,1.135876,0.983900,1.001446,1.000000,0.891304,1.0,1.007996,1.038462,0.984249,...,1.015278,1.038043,0.997520,1.005650,NaN,1.027204,0.996231,0.994449,0.996503,1.0
2013-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-07,1.025466,1.153289,0.960888,1.004800,0.996989,0.879447,1.0,1.003690,1.038462,0.979893,...,1.017598,1.038043,1.005521,1.014831,NaN,1.057431,0.993894,0.989611,1.154772,1.0
2013-01-08,0.995952,1.067775,0.948053,1.000175,0.996989,0.905138,1.0,1.003211,1.030769,0.980228,...,1.027687,1.036957,1.004950,1.023870,NaN,1.021663,0.992338,0.995781,1.154772,1.0
2013-01-09,1.002037,1.058197,0.953996,1.003352,1.000000,0.930830,1.0,1.001298,1.023077,0.979222,...,1.030332,1.029264,0.996198,1.010452,NaN,1.035265,0.989121,0.998803,1.154772,1.0


Trantando valores faltantes

In [11]:
for segmento in df_media_segm.columns:

    data_primeiro_ipo = df_media_segm[segmento].first_valid_index()
    # print(f"{segmento}, data_primeiro_ipo: {data_primeiro_ipo}")

    for data in df_media_segm.index:

        if data < data_primeiro_ipo:
            
            df_media_segm.loc[data, segmento] = 0

df_media_segm.drop(index="2013-01-01", inplace=True)
df_media_segm.dropna(inplace=True)
# df_media_segm.ffill(inplace=True)
df_media_segm.head(10)

,"Exploração, Refino e Distribuição",Equipamentos e Serviços,Minerais Metálicos,Siderurgia,Artefatos de Ferro e Aço,Artefatos de Cobre,Petroquímicos,Fertilizantes e Defensivos,Químicos Diversos,Madeira,...,Bancos,Soc. Crédito e Financiamento,Serviços Financeiros Diversos,Seguradoras,Resseguradoras,Corretoras de Seguros e Resseguros,Exploração de Imóveis,Intermediação Imobiliária,Holdings Diversificadas,Outros
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.0
2013-01-03,1.015322,1.028559,0.994528,1.010880,1.000000,0.936759,1.0,1.006082,1.000000,0.995308,...,1.014291,1.000000,1.010567,1.004237,0.0,1.012595,1.001772,1.001923,0.993357,1.0
2013-01-04,1.022497,1.135876,0.983900,1.001446,1.000000,0.891304,1.0,1.007996,1.038462,0.984249,...,1.015278,1.038043,0.997520,1.005650,0.0,1.027204,0.996231,0.994449,0.996503,1.0
2013-01-07,1.025466,1.153289,0.960888,1.004800,0.996989,0.879447,1.0,1.003690,1.038462,0.979893,...,1.017598,1.038043,1.005521,1.014831,0.0,1.057431,0.993894,0.989611,1.154772,1.0
2013-01-08,0.995952,1.067775,0.948053,1.000175,0.996989,0.905138,1.0,1.003211,1.030769,0.980228,...,1.027687,1.036957,1.004950,1.023870,0.0,1.021663,0.992338,0.995781,1.154772,1.0
2013-01-09,1.002037,1.058197,0.953996,1.003352,1.000000,0.930830,1.0,1.001298,1.023077,0.979222,...,1.030332,1.029264,0.996198,1.010452,0.0,1.035265,0.989121,0.998803,1.154772,1.0
2013-01-10,0.997594,1.034723,0.959591,0.995282,1.000000,0.936759,1.0,0.983254,1.023077,0.983914,...,1.028797,1.028177,0.999959,1.014831,0.0,1.020151,0.993884,0.993181,1.163513,1.0
2013-01-11,1.003524,1.052698,0.938056,1.271139,1.000000,0.936759,1.0,0.983732,1.023077,0.982574,...,1.033126,1.028177,0.998599,1.011441,0.0,1.057935,0.991524,1.004248,1.172254,1.0
2013-01-14,1.017268,1.065026,0.934609,1.270543,1.000000,0.954546,1.0,0.978469,1.023077,0.990617,...,1.032110,1.018395,1.005477,1.012288,0.0,1.085642,0.988930,0.996264,1.169107,1.0


In [12]:
df_media_segm.shape

(2671, 79)

Exportando df

In [13]:
df_media_segm.to_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\data\\var_media_segm.csv", index=True)